# 🎓 Campus Tour 3D Model Generator

This notebook converts campus videos to 3D Gaussian Splatting models using nerfstudio.

## Prerequisites
1. Upload your campus video to Google Drive
2. Update the VIDEO_PATH below
3. Run all cells

## GPU: Free T4 (16GB VRAM) - Better than most local GPUs!

In [ ]:
# @title ## 📹 Configuration
# @markdown ### Video path on Google Drive
VIDEO_PATH = "/content/drive/MyDrive/campus/tour_video.mp4"  # @param {type:"string"}
# @markdown ### Output folder name (no spaces)
OUTPUT_NAME = "cs_building"  # @param {type:"string"}
# @markdown ### Number of training iterations (more = better quality, slower)
ITERATIONS = 15000  # @param {type:"integer"}

In [ ]:
# @title ## 🚀 Step 1: Install Dependencies
!pip install -q nerfstudio
!pip install -q gsplat

In [ ]:
# @title ## 📂 Step 2: Setup Folders
import os

# Create output directories
BASE_DIR = "/content/campus_output"
DATA_DIR = os.path.join(BASE_DIR, "data", OUTPUT_NAME)
OUTPUT_DIR = os.path.join(BASE_DIR, "output", OUTPUT_NAME)

os.makedirs(DATA_DIR, exist_ok=True)
os.makedirs(OUTPUT_DIR, exist_ok=True)

print(f"📁 Data directory: {DATA_DIR}")
print(f"📁 Output directory: {OUTPUT_DIR}")

In [ ]:
# @title ## 📥 Step 3: Copy Video to Local
import shutil

if os.path.exists(VIDEO_PATH):
    local_video = os.path.join(DATA_DIR, "input_video.mp4")
    print(f"📥 Copying video from Google Drive...")
    shutil.copy2(VIDEO_PATH, local_video)
    print(f"✅ Video copied to: {local_video}")
    print(f"📊 Video size: {os.path.getsize(local_video) / (1024*1024):.1f} MB")
else:
    print("❌ Video not found! Please check VIDEO_PATH")
    print("Expected path:", VIDEO_PATH)

In [ ]:
# @title ## 🎨 Step 4: Process Video to Data
# @markdown This extracts frames and prepares data for training

import subprocess

cmd = [
    "ns-process-data",
    "video",
    "--data", DATA_DIR,
    "--output-dir", DATA_DIR,
    "--matching-method", "exhaustive",
    "--num-downscale", "2"
]

print("🎨 Processing video to data...")
print("This may take 5-15 minutes...")
result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout)
if result.returncode != 0:
    print("STDERR:", result.stderr)

In [ ]:
# @title ## 🧠 Step 5: Train Gaussian Splatting Model
# @markdown This trains the 3D model. Takes 15-45 minutes depending on iterations.

cmd = [
    "ns-train",
    "gaussian-splatting",
    "--data", DATA_DIR,
    "--output-dir", OUTPUT_DIR,
    "--max-num-iterations", str(ITERATIONS),
    "--pipeline.model.coarse-to-fine",
    "--viewer.enabled", "False"  # Disable viewer for Colab
]

print(f"🧠 Training Gaussian Splatting model...")
print(f"📊 Iterations: {ITERATIONS}")
print(f"⏱️ Estimated time: {ITERATIONS // 500} minutes")
print("=" * 50)

result = subprocess.run(cmd, capture_output=True, text=True)
print(result.stdout[-5000:] if len(result.stdout) > 5000 else result.stdout)
if result.returncode != 0:
    print("STDERR:", result.stderr[-2000:])

In [ ]:
# @title ## 📦 Step 6: Export to .ksplat Format
# @markdown This converts the trained model to .ksplat for web viewing

# Find the config file
config_path = None
for root, dirs, files in os.walk(OUTPUT_DIR):
    for f in files:
        if f.endswith('.yml') or f.endswith('.yaml'):
            config_path = os.path.join(root, f)
            break
    if config_path:
        break

if not config_path:
    print("❌ Config file not found!")
else:
    export_dir = os.path.join(OUTPUT_DIR, "exports")
    os.makedirs(export_dir, exist_ok=True)
    
    cmd = [
        "ns-export",
        "gaussian-splatting",
        "--load-config", config_path,
        "--output-dir", export_dir,
        "--export-format", "ply"
    ]
    
    print("📦 Exporting to .ply format...")
    result = subprocess.run(cmd, capture_output=True, text=True)
    print(result.stdout)
    
    # Find output files
    ply_files = []
    for f in os.listdir(export_dir):
        if f.endswith('.ply'):
            ply_files.append(os.path.join(export_dir, f))
    
    if ply_files:
        print(f"\n✅ Exported files:")
        for f in ply_files:
            size_mb = os.path.getsize(f) / (1024*1024)
            print(f"  📄 {f} ({size_mb:.1f} MB)")
    else:
        print("❌ No .ply files found")

In [ ]:
# @title ## ⬇️ Step 7: Download Models
# @markdown Run this cell to download your 3D model

from google.colab import files

# Find all PLY files
ply_files = []
for root, dirs, files in os.walk(OUTPUT_DIR):
    for f in files:
        if f.endswith('.ply'):
            ply_files.append(os.path.join(root, f))

print("📦 Available files for download:")
for f in ply_files:
    print(f"  - {f}")

if ply_files:
    print("\n⬇️ Click the download link below to download:")
    for f in ply_files:
        files.download(f)
else:
    print("❌ No files to download")

## 📋 How to Use This Notebook

### Step 1: Upload Video
1. Go to Google Drive
2. Create folder `campus`
3. Upload your campus video (MP4, MOV)
4. Note the path (e.g., `/content/drive/MyDrive/campus/video.mp4`)

### Step 2: Update Configuration
- Set `VIDEO_PATH` to your video location
- Set `OUTPUT_NAME` for the building name
- Set `ITERATIONS` (15000 = good quality, 30000 = best)

### Step 3: Run All Cells
- Run each cell in order
- Training takes 15-45 minutes

### Step 4: Download & Upload
1. Download the .ply file
2. Upload to GitHub: https://github.com/rajpal07/campus-assets
3. The app will automatically load it!

## 💡 Tips for Best Results
- **Video quality**: Use 4K or 1080p, 30-60fps
- **Movement**: Walk slowly, keep camera steady
- **Coverage**: Capture all angles of the building
- **Lighting**: Best during golden hour (soft light)
- **Avoid**: Motion blur, dark areas, reflections